In [1]:
!pip install neo4j pandas sentence-transformers langchain langchain-openai langchain-neo4j python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.7/201.7 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 28.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 8.5.0
    Uninstalling tenacity-8.5.0:
      Successfully uninstalled tenacity-8.5.0
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.75
    Uninstalling langchain-core-0.3.75:
      Successfully uninstalled langchain-core-0.3.75
ERROR: pi

In [2]:
! pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
google-adk 1.13.0 requires tenacity<9.0.0,>=8.0.0, but you have tenacity 9.1.2 which is incompatible.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.


In [3]:
!pip install fastapi uvicorn python-dotenv neo4j sentence-transformers langchain-openai

In [4]:
!pip install pyngrok

In [5]:
pip install --upgrade transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 123.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.1
    Uninstalling transformers-4.56.1:
      Successfully uninstalled transformers-4.56.1


In [7]:
import os
import pandas as pd
import numpy as np
from dotenv import load_dotenv
from neo4j import GraphDatabase
from sentence_transformers import SentenceTransformer
from langchain_openai import ChatOpenAI
from langchain_neo4j import Neo4jGraph, Neo4jVector
from langchain.prompts import ChatPromptTemplate
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from pyngrok import ngrok
import uvicorn
from threading import Thread
print('STARTING...')
# Load env
NGROK_AUTH_TOKEN = "330Gw6Neyfdx0Rhb2YLWVHGImJm_5cVmeVaj1NfQTmRX26VG3"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
load_dotenv()
NEO4J_URI = "neo4j+s://d59a45d3.databases.neo4j.io"          # neo4j+s://<id>.databases.neo4j.io
NEO4J_USER = "d59a45d3"    # Aura DB username (not Google SSO)
NEO4J_PASS = "YqinKDmv5DGGWTqauBkgCoER_hEdF-5n3MLrpE-YqsQ"  # Aura DB password
DB_NAME = "d59a45d3"
print(f"Connecting to Neo4j at {NEO4J_URI} as user {NEO4J_USER}")
# Connect (Aura requires TLS scheme neo4j+s)
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASS))  # [Aura TLS]
driver.verify_connectivity()

# Embedder
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Schema (Aura supports vector indexes + constraints)
schema = """
CREATE CONSTRAINT item_id IF NOT EXISTS FOR (i:Item) REQUIRE i.item_id IS UNIQUE;
CREATE CONSTRAINT seller_id IF NOT EXISTS FOR (s:Seller) REQUIRE s.seller_id IS UNIQUE;
CREATE CONSTRAINT cat_name IF NOT EXISTS FOR (c:Category) REQUIRE c.name IS UNIQUE;
CREATE CONSTRAINT brand_name IF NOT EXISTS FOR (b:Brand) REQUIRE b.name IS UNIQUE;
CREATE CONSTRAINT tag_name IF NOT EXISTS FOR (t:Tag) REQUIRE t.name IS UNIQUE;
CREATE CONSTRAINT coll_id IF NOT EXISTS FOR (c:Collection) REQUIRE c.collection_id IS UNIQUE;
CREATE CONSTRAINT user_id IF NOT EXISTS FOR (u:User) REQUIRE u.user_id IS UNIQUE;

CREATE VECTOR INDEX item_desc_embed IF NOT EXISTS
FOR (i:Item) ON (i.desc_vec)
OPTIONS {indexConfig: {`vector.dimensions`: 384, `vector.similarity_function`: "cosine"}};
"""
print("Setting up schema...")
with driver.session() as sess:
    for stmt in [s.strip() for s in schema.split(";") if s.strip()]:
        sess.run(stmt)

# # Load CSVs
# products = pd.read_csv("products.csv")
# sellers = pd.read_csv("sellers.csv")
# collections = pd.read_csv("collections.csv")
# collection_items = pd.read_csv("collection_items.csv")
# user_events = pd.read_csv("users.csv")

# LLM (your local/vLLM endpoint stays the same)
print("Setting up LLM...")
llm = ChatOpenAI(
    model="Qwen/Qwen2.5-1.5B-Instruct",         # or your local model id
    openai_api_key="EMPTY",              # vLLM can ignore/validate as configured
    openai_api_base="https://8081-01k5mf5j8b6hqtdxjy9wnhp2fy.cloudspaces.litng.ai/v1",
    temperature=0,
)
print("LLM setup complete.")
def upsert_products(df: pd.DataFrame):
    with driver.session() as sess:
        for _, r in df.iterrows():
            doc = f"{r.title}. {r.description}. Category: {r.category}. Brand: {r.brand}. Tags: {r.tags}"
            vec = embedder.encode([doc])[0].astype(np.float32).tolist()
            cypher = """
            MERGE (i:Item {item_id:$id})
              SET i.title=$title, i.description=$description, i.price=$price,
                  i.rating=$rating, i.popularity=$popularity, i.desc_vec=$vec
            MERGE (b:Brand {name:$brand})
            MERGE (c:Category {name:$category})
            MERGE (s:Seller {seller_id:$seller_id})
            MERGE (i)-[:OF_BRAND]->(b)
            MERGE (i)-[:IN_CATEGORY]->(c)
            MERGE (i)-[:SOLD_BY]->(s)
            WITH i
            UNWIND $tags AS tg
              MERGE (t:Tag {name:tg})
              MERGE (i)-[:HAS_TAG]->(t)
            """
            sess.run(cypher, parameters={
                "id": r.id,
                "title": r.title,
                "description": r.description,
                "price": int(r.price),
                "rating": float(r.rating),
                "popularity": float(r.popularity),
                "brand": r.brand,
                "category": r.category,
                "seller_id": r.seller_id,
                "tags": [t.strip() for t in str(r.tags).split(";") if t.strip()],
                "vec": vec,
            })

def upsert_sellers(df: pd.DataFrame):
    with driver.session() as sess:
        for _, r in df.iterrows():
            sess.run("""
            MERGE (s:Seller {seller_id:$seller_id})
              SET s.name=$name, s.region=$region
            """, parameters={"seller_id": r.seller_id, "name": r.name, "region": r.region})

def upsert_collections(cdf: pd.DataFrame, cidf: pd.DataFrame):
    with driver.session() as sess:
        for _, r in cdf.iterrows():
            sess.run("""
            MERGE (c:Collection {collection_id:$cid})
              SET c.name=$name, c.description=$desc
            """, parameters={"cid": r.collection_id, "name": r.name, "desc": r.description})
        for _, r in cidf.iterrows():
            sess.run("""
            MATCH (c:Collection {collection_id:$cid}), (i:Item {item_id:$iid})
            MERGE (c)-[:HAS_ITEM]->(i)
            """, parameters={"cid": r.collection_id, "iid": r.item_id})


# def upsert_user_events(df: pd.DataFrame):
#   required = {"user_id", "name", "region", "preferred_categories", "budget_band"}
#   missing = required - set(df.columns)
#   if mitr(r["preferred_categories"]),
#       "band": r["budget_band"],
#       )

# def create_similssing():
#     raise ValueError(f"users.csv missing columns: {missing}")
#   with driver.session() as sess:
#     for _, r in df.iterrows():
#       sess.run("""
#       MERGE (u:User {user_id:$u})
#       SET u.name=$name,
#       u.region=$region,
#       u.preferred_categories=$cats,
#       u.budget_band=$band
#       """, parameters={
#       "u": r["user_id"],
#       "name": r["name"],
#       "region": r["region"],
#       "cats": sarity(k=3):
#     with driver.session() as sess:
#         sess.run("""
#         MATCH (i:Item)
#         WITH collect(i) AS items
#         UNWIND items AS i1
#         WITH i1, [i IN items WHERE i <> i1] AS others
#         UNWIND others AS i2
#         WITH i1, i2, vector.similarity.cosine(i1.desc_vec, i2.desc_vec) AS sim
#         ORDER BY i1.item_id, sim DESC
#         WITH i1, collect({n:i2, s:sim})[0..$k] AS nbrs
#         UNWIND nbrs AS nb
#         WITH i1, nb.n AS i2, nb.s AS sim
#         MERGE (i1)-[r:SIMILAR_TO]->(i2)
#         SET r.score = sim
#         """, parameters={"k": k})  # [web:45][web:15][web:49]
print("Ingesting data...")
# Ingest
# upsert_sellers(sellers)
# upsert_products(products)
# upsert_collections(collections, collection_items)
# upsert_user_events(user_events)
# create_similarity(k=3)
print("Data ingestion complete.")
# Retrieval utilities (Aura connection reused)
graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USER, password=NEO4J_PASS, database=DB_NAME )

from langchain_community.embeddings import HuggingFaceEmbeddings

emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vector = Neo4jVector.from_existing_graph(
    embedding=emb,
    url=NEO4J_URI,
    username=NEO4J_USER,
    password=NEO4J_PASS,
    database=DB_NAME,
    node_label="Item",
    embedding_node_property="desc_vec",
    text_node_properties=["title","description"],
    index_name="item_desc_embed",
)

route_prompt = ChatPromptTemplate.from_template(
    "Classify intent for recommendation: '{q}'. Reply exactly 'GRAPH' if constraints/categories/tags/collections dominate; else 'VECTOR'."
)
print("Setting up routing...")
def route(q: str) -> str:
    resp = llm.invoke(route_prompt.format_messages(q=q))
    t = resp.content.strip().upper()
    return "GRAPH" if "GRAPH" in t else "VECTOR"

def graph_candidates(user_id: str, q: str, limit: int = 20):
    cypher = """
    // Collect Father's Day collection items; cast name to string before lower
    OPTIONAL MATCH (col:Collection)-[:HAS_ITEM]->(i:Item)
    WHERE toLower(toString(col.name)) CONTAINS toLower('father')
    WITH collect(DISTINCT i) AS col_items

    // Collect items by tags; cast tag.name to string and filter out nulls
    OPTIONAL MATCH (i2:Item)-[:HAS_TAG]->(t:Tag)
    WHERE t.name IS NOT NULL AND toLower(toString(t.name)) IN ['gift','father','men']
    WITH col_items, collect(DISTINCT i2) AS tag_items

    // Concatenate lists and deduplicate without implicit grouping
    WITH col_items + tag_items AS lists
    UNWIND lists AS lst
    UNWIND lst AS c
    WITH DISTINCT c

    // Personalize and score (no string casts needed here)
    OPTIONAL MATCH (:User {user_id:$uid})-[:VIEWED|PURCHASED]->(h:Item)
    OPTIONAL MATCH (h)-[:HAS_TAG]->(ht)<-[:HAS_TAG]-(c)
    OPTIONAL MATCH (h)-[:IN_CATEGORY]->(hc)<-[:IN_CATEGORY]-(c)
    WITH c, count(DISTINCT ht)+count(DISTINCT hc) AS overlap
    WITH c, (0.6*overlap + 0.25*coalesce(c.popularity,0) + 0.15*coalesce(c.rating,0)) AS score
    RETURN c.item_id AS item_id, c.title AS title, c.description AS description, c.price AS price, score
    ORDER BY score DESC LIMIT $limit
    """
    with driver.session() as sess:
        return sess.run(cypher, parameters={"uid": user_id, "limit": limit}).data()



def vector_candidates(q: str, limit: int = 20):
    docs = vector.similarity_search_with_score(q, k=limit)
    out = []
    for d, s in docs:
        out.append({
            "item_id": d.metadata.get("item_id"),
            "title": d.metadata.get("title") or d.page_content[:60],
            "description": d.page_content,
            "price": d.metadata.get("price"),
            "score": float(s)
        })
    return out

def explain_paths(user_id: str, item_id: str, k: int = 2):
    cypher = """
    MATCH (u:User {user_id:$u}), (i:Item {item_id:$i})
    OPTIONAL MATCH p1=(u)-[:VIEWED|PURCHASED]->(:Item)-[:SIMILAR_TO]->(i)
    OPTIONAL MATCH p2=(u)-[:VIEWED|PURCHASED]->(:Item)-[:HAS_TAG]->(:Tag)<-[:HAS_TAG]-(i)
    OPTIONAL MATCH p3=(:Collection)-[:HAS_ITEM]->(i)
    WITH [p IN [p1,p2,p3] WHERE p IS NOT NULL][..$k] AS paths
    RETURN paths
    """
    with driver.session() as sess:
        res = sess.run(cypher, parameters={"u": user_id, "i": item_id, "k": k}).data()
    return res[0]["paths"] if res else []

def recommend_for_query(user_id: str, query: str, top_k: int = 5):
    mode = route(query)
    cands = graph_candidates(user_id, query, 30) if mode == "GRAPH" else vector_candidates(query, 30)
    sel_prompt = ChatPromptTemplate.from_template(
        "Intent: {q}\nCandidates: {c}\nPick top {k} items covering diverse categories and budgets; "
        "prefer tags relevant with intent; output JSON list with item_id and a one-sentence reason."
    )
    selection = llm.invoke(sel_prompt.format_messages(q=query, c=str(cands), k=top_k)).content
    import json
    try:
        picks = json.loads(selection)
    except Exception:
        picks = []
    enriched = []
    for p in picks[:top_k]:
        paths = explain_paths(user_id, p.get("item_id"))
        enriched.append({**p, "paths": str(paths)[:400]})
    final_prompt = ChatPromptTemplate.from_template(
        "Intent: {q}\nItems: {items}\nWrite a concise recommendation list (bullet points): "
        "name, price, reason; include a short explanation when paths indicate a match with tags/collection."
    )
    return llm.invoke(final_prompt.format_messages(q=query, items=str(enriched))).content
print("Ready for recommendations.")
print(recommend_for_query("U100", "I want to buy gift for fathers day", top_k=5))

app = FastAPI()
origins = ["*"] # or specify your frontend's domain, e.g., "http://127.0.0.1:5500"

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)
class RecommendationRequest(BaseModel):
    user_id: str
    query: str
    top_k: int = 5

@app.post("/recommend")
def recommend(request: RecommendationRequest):
  try:
      recommendations = recommend_for_query(
          user_id=request.user_id,
          query=request.query,
          top_k=request.top_k
          )
      return {"recommendation": recommendations}
  except Exception as e:
      return {"error": str(e)},500
@app.get("/health")
def health_check():
    return {"status": "ok"}

def run_server():
  uvicorn.run(app, host="0.0.0.0", port=8000)

Thread(target=run_server).start()
public_url = ngrok.connect(8000)
print("Ngrok Tunnel URL: {public_url}")
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:8000\"".format(public_url))

STARTING...
Connecting to Neo4j at neo4j+s://d59a45d3.databases.neo4j.io as user d59a45d3
Setting up schema...
Setting up LLM...
LLM setup complete.
Ingesting data...
Data ingestion complete.
Setting up routing...
Ready for recommendations.


- 1. Father's Day Special Gift - $50 - A personalized wooden clock that tells the time in both English and French, perfect for showing your father how much you appreciate his bilingual skills.
- 2. Father's Day Gourmet Basket - $75 - A gourmet basket filled with artisanal cheeses, chocolates, and精选的葡萄酒，适合父亲节送给他一份精致的美食体验。
- 3. Father's Day Classic Book Bundle - $40 - A collection of classic books on various topics such as history, science, and literature, ideal for someone who loves reading.
- 4. Father's Day Tech Gift - $90 - An advanced smartwatch with fitness tracking features, perfect for keeping track of your father's health and wellness goals.
- 5. Father's Day Personalized Jewelry Set - $60 - A set of personalized necklaces and bracelets featuring your father's initials or favorite quotes, making it a thoughtful and unique gift.


INFO:     Started server process [700]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): [errno 98] address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


Ngrok Tunnel URL: {public_url}
 * ngrok tunnel "NgrokTunnel: "https://117f0ef49ffa.ngrok-free.app" -> "http://localhost:8000"" -> "http://127.0.0.1:8000"


In [ ]:
!curl -X POST "https://cc120e6fd6e7.ngrok-free.app/recommend" \
-H "Content-Type: application/json" \
-d '{"user_id": "U100","query": "I want to buy a gift for fathers day","top_k": 5}'

INFO:     34.125.54.86:0 - "POST /recommend HTTP/1.1" 200 OK
{"recommendation":"- 1. Father's Day Gift Basket - $25 - A curated basket filled with thoughtful items like gourmet chocolates, candles, and personalized cards.\n- 2. Leather Wallet Set - $30 - A durable leather wallet set that is perfect for any father who appreciates quality craftsmanship.\n- 3. Personalized Father's Day Card - $10 - A heartfelt card designed specifically for the recipient, featuring their favorite quote or message.\n- 4. High-Quality Handmade Soap Bar - $15 - A luxurious soap bar made from natural ingredients, ideal for those looking for something unique and eco-friendly.\n- 5. Customizable Wooden Clock - $20 - A stylish wooden clock that can be customized with your own personal touches, making it a thoughtful gift for any occasion."}